In [1]:
#importing basic libraries
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

In [2]:
#importing dataset
cars = pd.DataFrame()

In [3]:
filename = 'imports-85.data'

In [4]:
names = ['symboling', 'normalized-losses', 'make','fuel-type','aspiration','num-of-doors','body-style','drive-wheels','engine-location',
        'wheel-base','length', 'width', 'height', 'curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore','stroke',
        'compression-ratio','horsepower','peak-rpm', 'city-mpg','highway-mpg','price']


In [5]:
cars = pd.read_csv(filename ,names = names , delimiter=',')

In [6]:
cars.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [7]:
#there is missing values and those fields are filled with '?'. Hence most of the column are of type object
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
symboling            205 non-null int64
normalized-losses    205 non-null object
make                 205 non-null object
fuel-type            205 non-null object
aspiration           205 non-null object
num-of-doors         205 non-null object
body-style           205 non-null object
drive-wheels         205 non-null object
engine-location      205 non-null object
wheel-base           205 non-null float64
length               205 non-null float64
width                205 non-null float64
height               205 non-null float64
curb-weight          205 non-null int64
engine-type          205 non-null object
num-of-cylinders     205 non-null object
engine-size          205 non-null int64
fuel-system          205 non-null object
bore                 205 non-null object
stroke               205 non-null object
compression-ratio    205 non-null float64
horsepower           205 non-nul

In [8]:
#importing dataset with na_values denoted by ?. Displaying value counts for all object type columns for further analysis
cars = pd.read_csv(filename ,names = names , delimiter=',', na_values="?")
print(cars.info())
print('make:\n',cars['make'].value_counts())
print('fuel-type:\n',cars['fuel-type'].value_counts())
print('aspiration:\n',cars['aspiration'].value_counts())
print('num-of-doors:\n',cars['num-of-doors'].value_counts())
print('body-style:\n',cars['body-style'].value_counts())
print('drive-wheels:\n',cars['drive-wheels'].value_counts())
print('engine-location:\n',cars['engine-location'].value_counts())
print('engine-type:\n',cars['engine-type'].value_counts())
print('num-of-cylinders:\n',cars['num-of-cylinders'].value_counts())
print('fuel-system:\n',cars['fuel-system'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
symboling            205 non-null int64
normalized-losses    164 non-null float64
make                 205 non-null object
fuel-type            205 non-null object
aspiration           205 non-null object
num-of-doors         203 non-null object
body-style           205 non-null object
drive-wheels         205 non-null object
engine-location      205 non-null object
wheel-base           205 non-null float64
length               205 non-null float64
width                205 non-null float64
height               205 non-null float64
curb-weight          205 non-null int64
engine-type          205 non-null object
num-of-cylinders     205 non-null object
engine-size          205 non-null int64
fuel-system          205 non-null object
bore                 201 non-null float64
stroke               201 non-null float64
compression-ratio    205 non-null float64
horsepower           203 non-

In [9]:
#separating label, category columns and numerical columns to work on them
cars_cat = cars.select_dtypes(include=['object']).copy()
cars_num = cars.drop(columns = cars_cat,axis=1)
cars_label = cars['symboling'].copy()

In [10]:
#displaying null values in category columns
cars_cat[cars_cat.isnull().any(axis=1)]

,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,engine-type,num-of-cylinders,fuel-system
27,dodge,gas,turbo,NaN,sedan,fwd,front,ohc,four,mpfi
63,mazda,diesel,std,NaN,sedan,fwd,front,ohc,four,idi


In [11]:
#filling with categorical values
cars_cat = cars_cat.fillna({"num-of-doors": "four"})

In [12]:
#imputing values with Random forest for numerical columns one at a time
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

rfr = RandomForestRegressor(n_estimators=100)
rfc = RandomForestClassifier(n_estimators=100)

X_raw = cars_num.loc[:,"normalized-losses":]
X_imputed = cars_num.loc[:,"normalized-losses":]

for column in X_raw:
    Y = X_raw.loc[:, column]
    X = X_raw.drop(column, axis=1).fillna(-99)
    
    # ignore columns without missing values
    if Y.isnull().sum() == 0:
        continue  
        
    X_train = X[~Y.isnull()]
    Y_train = Y[~Y.isnull()]
    X_test = X[Y.isnull()]
     # use classifier or regressor
    if np.array_equal(Y_train, Y_train.astype("int")):
        Y_predicted = rfc.fit(X_train, Y_train).predict(X_test)
    else:
        Y_predicted = rfr.fit(X_train, Y_train).predict(X_test)
    
    # use '.loc[]' to write data into DataFrame
    X_imputed.loc[Y.isnull(), column] = Y_predicted 

In [13]:
X_imputed.head()

,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,168.0,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,168.0,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,150.0,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,164.0,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,164.0,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [14]:
X_imputed.isnull().sum()

normalized-losses    0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-size          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
dtype: int64

In [15]:
#label encoding for all categorical columns and concatenating all the dataframes into the final dataset
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
le = preprocessing.LabelEncoder()
cars_cat_encoded = cars_cat.apply(le.fit_transform)
cars_final = pd.concat([X_imputed,cars_cat_encoded,cars_label], axis=1 )
cars_final.head()

,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,compression-ratio,...,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,engine-type,num-of-cylinders,fuel-system,symboling
0,168.0,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,1,0,1,0,2,0,0,2,5,3
1,168.0,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,1,0,1,0,2,0,0,2,5,3
2,150.0,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,...,1,0,1,2,2,0,5,3,5,1
3,164.0,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,...,1,0,0,3,1,0,3,2,5,2
4,164.0,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,...,1,0,0,3,0,0,3,1,5,2


In [16]:
#verifying for mising values
cars_final.isnull().sum()

normalized-losses    0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-size          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
make                 0
fuel-type            0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
engine-type          0
num-of-cylinders     0
fuel-system          0
symboling            0
dtype: int64

In [17]:
#importing neccasary libraries
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeClassifier

In [18]:
#cars_data is the input features
cars_data = np.array(cars_final.drop(['symboling'],1))
#cars_target is the target 
cars_target = np.array(cars_final['symboling'])

In [19]:
#train test split
train_data, test_data,train_target, test_target = train_test_split(cars_data,cars_target,test_size=0.33, random_state=0)

In [20]:
#modelling pipe
pipe = Pipeline([
  ('feature_selection', SelectFromModel(RandomForestClassifier(n_estimators=100, max_features=0.5,random_state=47))),
   ('transformation',  QuantileTransformer(n_quantiles=100,output_distribution="uniform" , random_state=47)),
    ('classification', RidgeClassifier(tol= 0.0001,random_state=47))
])

In [21]:
#defining param dict
param_grid = [
    {'classification__alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10]}
    ]

In [22]:
#defining gridsearchcv
model = GridSearchCV(pipe,param_grid,cv=5,n_jobs=-1,verbose=10)

In [23]:
#fitting model
cars_grid = model.fit(train_data,train_target)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:    4.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    4.5s finished
C:\Users\Harini Pravin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [24]:
print("Best cross-validation score: {:.2f}".format(cars_grid.best_score_))

Best cross-validation score: 0.66


In [25]:
pred = cars_grid.predict(test_data)

In [26]:
#model accuracy
acc = cars_grid.score(test_data,test_target)
print("Accuracy: %f" % (acc))

Accuracy: 0.544118
